1. Artificial Intelligence A Modern Approach (3rd Edition).pdf
2. http://what-when-how.com/artificial-intelligence/a-comparison-of-cooling-schedules-for-simulated-annealing-artificial-intelligence/

### Hill-climbing

In [52]:
# Psuedo Code
def hill_climbing(problem):
    current_state = problem.initial_state
    while(True):
        neighbor = max(current_state.neighbors)
        if neighbor.value < current_state.value:
            return current_state
        else:
            current_state = neighbor

1. Does not look beyond immediate neigbors.
2. Result dependent on initial state and state space.
3. The algorithm is prone to getting stuck at local maxima and is weak against "ridges".[1]
4. Sometimes called greedy local search, as never makes "downhill" move, and in the context of exploration vs. exploitation, is only exploiting.
5. To address local maxima issue, variants have been designed such as stochastic hill climbing, first-choice hill climbing, random-restart hill climbing.


### Simulated Annealing

"To explain simulated annealing,  we switch our point of view from hill climbing to
gradient descent (i.e., minimizing cost) and imagine the task of getting a gradient descent ping-pong ball into the deepest crevice in a bumpy surface.  If we just let the ball roll, it will
come to rest at a local minimum.  If we shake the surface, we can bounce the ball out of the
local minimum.  The trick is to shake just hard enough to bounce the ball out of local min-
ima but not hard enough to dislodge it from the global minimum.  The simulated-annealing
solution is to start by shaking hard (i.e., at a high temperature) and then gradually reduce the
intensity of the shaking (i.e., lower the temperature)." - AIMA

In [54]:
# Psuedo Code
def simulated_annealing(problem, schedule):
    """
    Args:
        problem - the problem
        schedule - a mapping from time t, to temperature T
    Returns:
        state - solution state.
    """
    current_state = problem.initial_state
    while (True):
        T = schedule(t)
        if T == 0:
            return current_state
        else:
            next_state = random_select(current_state.neighbors)
            delta_E = next_state.value - current_state.value
            if delta_E > 0:
                current_state = next_state
            elif random.normal() > exp(delta_E/T): 
                # Exploration: Will take worse path with small probability.
                current_state = next_state

1. Tries to combine hill climbing (efficient but incomplete) with a random walk (inefficient but complete).
2. Can be considered a stochastic version of hill climbing with some moves down, allows for some exploration.
3. The "cooling" schedule is stated to be the most important factor in the algorithm. Some suggestions for annealing schedules are presented below. An indepth summary can be found at [2]

Exponential multiplicative cooling: \\[T_k = T_0\alpha^k \ (0.8 \leq \alpha 0.9) \\]

Logarithmical multiplicative cooling: \\[T_k = \frac{T_0}{1+\alpha k Log(1+k)} \ (\alpha > 1) \\]

Linear multiplicative cooling: \\[T_k = \frac{T_0}{1+\alpha k} \ (\alpha > 0) \\]

Quadratic multiplicative cooling: \\[T_k = \frac{T_0}{1+\alpha k^2} \ (\alpha > 0) \\]

### Local Beam Search

In [11]:
def local_beam_search(problem):
    (state_0, state_1,state_k) = problem.initial_k_states
    states = (state_0, state_1,state_k)
    while True:
        for state in states:
            if termination_condition(state.neighbors) == True:
                return "neighbor that meets condition"
            else:
                all_states.append(state.neighbors)

            states = select_k_max(all_states)


1. Introduces the expanded memory concept. As opposed to storing the last state, beam search stores k states.
2. Generates sucessors for each of the k states, and takes the best k states if none represents termination.
3. Can suffer from a lack of diversity among k-states as they can quickly become concentrated in a small region of the state space.

### Genetic Algorithms

Fitness -> Selection -> Pairs -> Cross-Over -> Mutation
1. The fitness function is used to evaluates the sample.
2. A probability distribution of the samples based on fitness is used to calculate the likelihood of sample becoming a "parent".
3. A split is selected randomly, and cross-over is used to create the offsping.
4. GA mutation is used to infuse randomness to the offspring.
5. Evolution is a natural dampening mechanism of randomness as resulting offspring output similar fitness functions and weights become more uniform.
6. A variant of stochastic beam search, but not by asexual reproduction but by sexual reproduction, ie, the result of two parents. [1]

An example using the genetic algorithm:

Consider a genetic algorithm in which individuals are represented using a 5-bit string of the form b1b2b3b4b5. An example of an individual is 00101, for which b1 = 0, b2 = 0, b3 = 1, b4 = 0, b5 = 1. The fitness function is defined over these individuals as follows:

 
f(b1b2b3b4b5) = b1 + b2 + b3 + b4 + b5 + AND(b1, b2, b3, b4, b5)

 
where


AND(b1, b2, b3, b4, b5) = 1, if b1 = b2 = b3 = b4 = b5 = 1

AND(b1, b2, b3, b4, b5) = 0, otherwise.

In [49]:
import numpy as np

class GeneticAlgo(object):
    
    def __init__(self, init_population, cross_over=3, mutate_idx=None):
        self._parents = self._convert_string(init_population)
        self._cross_over = cross_over
        self._mutate_idx = mutate_idx
        self._pop_size = len(self._parents)

    def _convert_string(self, population):
        parents = []
        for individual in population:
            bit_str = list(individual)
            parent = [int(bit) for bit in bit_str]
            parents.append(parent)
        return parents

    def update_population(self, population):
        self._parents = population
    
    def get_parents(self, selection_probs):
        parent1 = self._parents[np.random.choice(self._pop_size, 1, p=selection_probs)[0]]
        parent2 = self._parents[np.random.choice(self._pop_size, 1, p=selection_probs)[0]]
        return parent1, parent2

    def get_child(self, parent1, parent2):
        child = parent1[:self._cross_over] + parent2[self._cross_over:]
        return child

    def get_fit_values(self):
        return [self.fitness_fn(x) for x in self._parents]

    def mutate(self,child):
        if self._mutate_idx is not None:
            i = self._mutate_idx
        else:
            i = np.random.choice(5,1)[0]-1
        child[i] = 1 ^ child[i]
        return child
    
    @property
    def pop_size(self):
        return self._pop_size
    
    @staticmethod
    def fitness_fn(parent):
        return sum(parent) + np.all(list(map(lambda bit: bit == 1, parent)))
    
    @staticmethod
    def selection_probs(fit_values):
        return fit_values/np.sum(fit_values)

In [50]:
individuals = ['00100', '11000', '01001', '10010', '00100']
gen = GeneticAlgo(individuals)

The optimal solution is a 11111, with a fitness value of 6. As there is a small chance of mutation, we see the algorithm converge to a value shy of 6.

In [55]:
epoch_fitness = []
for epoch in range(10):
    fitness = []
    for step in range(10000):
        fit_values = gen.get_fit_values()
        fitness.append(np.mean(fit_values))
        selection_prob = gen.selection_probs(fit_values)
        children = []
        for i in range(gen.pop_size):
            parent1, parent2 = gen.get_parents(selection_prob)
            child = gen.get_child(parent1, parent2)
            if np.random.uniform() < 0.001:
                child = gen.mutate(child)
                
            children.append(child)
        gen.update_population(children)
    print(np.mean(fitness))
    epoch_fitness.append(np.mean(fitness))

5.9783
4.89214
4.637580000000001
5.9927
5.57824
5.64364
5.538360000000001
4.7908800000000005
5.1648
5.9952
